In [1]:
# !pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/70/a9/96627b82b883f6b9f6a5dcbcc1571d57bd5696f69846f2df78a42b46f18f/boto3-1.34.115-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.35.0,>=1.34.115 from https://files.pythonhosted.org/packages/2e/6d/1c4302be80ae8c9f772173d4734f956dbcd7625bb628406d8aed8705c6c3/botocore-1.34.115-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/83/37/395cdb6ee92925fa211e55d8f07b9f93cf93f60d7d4ce5e66fd73f1ea986/s3transfer-0.10.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/139.3 kB ? eta -:--:--
   -------- ------------------------------ 30.7/139.3 kB 640.0 kB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import boto3

In [2]:
import psycopg2
import pandas as pd
import json

In [31]:
KEY = 'Enter your key here'
SECRET = 'Enter your secret key here'

DWH_DB = 'dev'
DWH_DB_USER = 'awsuser'
DWH_DB_PASSWORD = 'Enter your password here'
DWH_PORT = 5439
DWH_IAM_ROLE_NAME = 'redshift-s3-access'

REDSHIFT_HOST_ENDPOINT = 'Enter your Redshift endpoint here'

REGION_NAME = 'eu-west-2'

In [6]:
s3 = boto3.resource('s3',
                   region_name=REGION_NAME,
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

In [15]:
iam = boto3.client('iam',
                      region_name=REGION_NAME,
                      aws_access_key_id=KEY,
                      aws_secret_access_key=SECRET)

In [20]:
redshift = boto3.client('redshift-serverless',
                        region_name=REGION_NAME,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

In [7]:
bucket = s3.Bucket("Enter your bucket name here")

In [13]:
data_files = [fname.key for fname in bucket.objects.filter()]
data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [18]:
role_arn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
role_arn

'arn:aws:iam::905418467982:role/redshift-s3-access'

In [33]:
try:
    conn = psycopg2.connect(host=REDSHIFT_HOST_ENDPOINT,
                           dbname=DWH_DB,
                           user=DWH_DB_USER,
                           password=DWH_DB_PASSWORD,
                           port=DWH_PORT)
except Exception as e:
    print(e)

conn.set_session(autocommit=True)

In [34]:
try:
    cur = conn.cursor()
except Exception as e:
    print(e)

In [35]:
cur.execute("""
    CREATE TABLE students (
    id INT,
    name VARCHAR,
    age INT
    );
""")

In [36]:
cur.execute("INSERT INTO students VALUES (1, 'Ganesh', 25);")
cur.execute("INSERT INTO students VALUES (2, 'Harri', 26);")

In [37]:
cur.execute("SELECT * FROM students;")

In [38]:
rows = cur.fetchall()
for i in rows:
    print(i)

(1, 'Ganesh', 25)
(2, 'Harri', 26)


In [39]:
conn.close()